In [1]:
!onmt_translate \
  -model models/model_step_10000.pt \
  -src input.txt \
  -output output.txt \
  -beam_size 5 \
  -batch_size 32 \
  --src_subword_model sentencepiece.model \
  --tgt_subword_model sentencepiece.model \
  --gpu 0


Time w/o python interpreter load/terminate:  3.134254217147827


C:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:46: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
C:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
C:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:395: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
C:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:444: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
[2025-07-16 15:37:13,856 INFO] Loading checkpoint from models/model_step_10000.pt
[2025-

In [ ]:
scores, predictions = global_translator.translate_batch(batch)

print("Raw predictions:", predictions)

if predictions and len(predictions[0]) > 0:
    pred_ids = predictions[0][0]
    print("Predicted token IDs:", pred_ids)

    if isinstance(pred_ids, torch.Tensor):
        pred_ids = pred_ids.cpu().tolist()

    # 토큰을 스페셜 토큰 제외하고 출력해보기
    special_ids = {translator._tgt_pad_idx, global_translator._tgt_bos_idx,
                   translator._tgt_eos_idx, global_translator._tgt_unk_idx}
    filtered_ids = [tid for tid in pred_ids if tid not in special_ids]
    print("Filtered token IDs:", filtered_ids)

    # 디코딩 시도
    decoded_text = sp.decode_ids(filtered_ids)
    print("Decoded text:", decoded_text)


In [2]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load("sentencepiece.model")

text = "안녕하세요"
tokens = sp.encode(text, out_type=int)
print("Tokens:", tokens)
print("Tokens as pieces:", sp.encode(text, out_type=str))
print("Vocab size:", sp.get_piece_size())

Tokens: [23303, 19608]
Tokens as pieces: ['▁안녕', '하세요']
Vocab size: 30000


In [1]:
from model_class import KoreanEnglishTranslator

c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:46: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:395: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:444: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [2]:
global_translator = KoreanEnglishTranslator(
    model_path='models/model_step_10000.pt',
    tokenizer_path='sentencepiece.model',
)

In [3]:
predicted_english = global_translator.translate("반가워요 친구들")
print(predicted_english)

tokens: [610, 28911, 29297, 29060, 6934, 28978]
src_tensor.shape: torch.Size([1, 6, 1])
src_len.shape: torch.Size([1]), src_len: tensor([6])
✅ [테스트] embedding output shape: torch.Size([1, 6, 500])
🔍 [run_encoder] src.shape: torch.Size([1, 6, 1])
🔍 [run_encoder] src_len: tensor([6], device='cuda:0')
[RNNEncoder.forward] input src shape: torch.Size([1, 6, 1])
[RNNEncoder.forward] after embeddings shape: torch.Size([1, 6, 500])
[RNNEncoder.forward] packed_emb shape: PackedSequence
[RNNEncoder.forward] enc_out shape: PackedSequence
[RNNEncoder.forward] unpacked enc_out shape: torch.Size([1, 6, 500])
✅ [run_encoder] Encoder ran successfully.
🔍 [init_state] src.shape: torch.Size([1, 6, 1])
🔍 [init_state] type(enc_final_hs): <class 'tuple'>
  🔸 h_n shape: torch.Size([2, 1, 500])
  🔸 c_n shape: torch.Size([2, 1, 500])
✅ [init_state] hidden state initialized with batch size: 1
Translation error: 'coverage'



In [4]:
print("encoder:", type(global_translator.translator.model.encoder))
print("encoder.rnn:", global_translator.translator.model.encoder.rnn)

# batch_first 값 확인
print("batch_first:", global_translator.translator.model.encoder.rnn.batch_first)

encoder: <class 'onmt.encoders.rnn_encoder.RNNEncoder'>
encoder.rnn: LSTM(500, 500, num_layers=2, batch_first=True, dropout=0.3)
batch_first: True


In [5]:
print("encoder:", type(global_translator.translator.model.decoder))
print("encoder.rnn:", global_translator.translator.model.decoder.rnn)

# batch_first 값 확인
# print("batch_first:", global_translator.translator.model.decoder.rnn.batch_first)

encoder: <class 'onmt.decoders.decoder.InputFeedRNNDecoder'>
encoder.rnn: StackedLSTM(
  (dropout): Dropout(p=0.3, inplace=False)
  (layers): ModuleList(
    (0): LSTMCell(1000, 500)
    (1): LSTMCell(500, 500)
  )
)


In [ ]:
emb = global_translator.embeddings(src)
print("[RNNEncoder.forward] embedding output shape:", emb.shape)